In [1]:
from rnn import *
from fixed_points import *
from viz import *
from utils import *

In [2]:
batch_size = 1
n_epochs = 1000
bits = 1

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Create task
task = FlipFlopTask(batch_size=batch_size, T=100, dt=1.0, flip_prob=0.05, bits=bits)
U, _ = task.get()

# Create RNN
rnn = RNN(input_size=task.bits, hidden_size=100, output_size=task.bits, tau=2.0, dt=1.0, use_bias=True)

In [3]:
# Train the RNN
train_R_list, losses, weight_snapshots = train_rnn(rnn, task, n_epochs=n_epochs, learning_rate=1e-2, batch_size=batch_size)
train_R_list = process_R_list(train_R_list)

Training RNN on flip flop task...
Epochs: 1000, Learning rate: 0.01, Batch size: 1
------------------------------------------------------------
Epoch 100/1000, Loss: 0.272948
Epoch 200/1000, Loss: 0.080789
Epoch 300/1000, Loss: 0.026353
Epoch 400/1000, Loss: 0.128781
Epoch 500/1000, Loss: 0.042447
Epoch 600/1000, Loss: 0.022009
Epoch 700/1000, Loss: 0.010337
Epoch 800/1000, Loss: 0.007623
Epoch 900/1000, Loss: 0.002557
Epoch 1000/1000, Loss: 0.001583
------------------------------------------------------------
Training complete!
Shape of R_list: (1000, 1, 101, 100)
Shape of R_list after processing: (1000, 101, 100)


In [4]:
# animate fixed points over training, using PCA fitted on all fixed points found across training
_ = animate_fixed_points(rnn, train_R_list, save_path="plots/FP_training1_fullpca.mp4", stride=10, weight_snapshots=weight_snapshots)

Finding fixed points for 1000 epochs...
  Epoch 100/1000: 1 fixed points
  Epoch 200/1000: 1 fixed points
  Epoch 300/1000: 1 fixed points
  Epoch 400/1000: 1 fixed points
  Epoch 500/1000: 0 fixed points
  Epoch 600/1000: 0 fixed points
  Epoch 700/1000: 0 fixed points
  Epoch 800/1000: 0 fixed points
  Epoch 900/1000: 1 fixed points
  Epoch 1000/1000: 0 fixed points
Generating animation with 100 frames...
Saving animation to plots/FP_training1_fullpca.mp4...
Animation saved successfully!


In [5]:
# # this is incorrect because it uses the current rnn weight to find fixed points, rather than the weights at the time
# unique_fps = find_fixed_points_KE_min(rnn, train_R_list[80])
# pca, lim = animate_R(train_R_list[80], save_path="plots/train_1.mp4", fps=30, stride=1, fixed_points=unique_fps, title="RNN trajectory, first training", pca=pca)

In [6]:
# animate fixed points over inference
R_list, _, weight_snapshots = inference_rnn(rnn, task)
R_list = process_R_list(R_list)
_ = animate_fixed_points(rnn, R_list, save_path="plots/FP_inference1_fullpca.mp4", stride=10, weight_snapshots=weight_snapshots)

Running inference...
------------------------------------------------------------
Epoch 100/1000, Loss: 0.001186
Epoch 200/1000, Loss: 0.001123
Epoch 300/1000, Loss: 0.001419
Epoch 400/1000, Loss: 0.000827
Epoch 500/1000, Loss: 0.001288
Epoch 600/1000, Loss: 0.001152
Epoch 700/1000, Loss: 0.001635
Epoch 800/1000, Loss: 0.001688
Epoch 900/1000, Loss: 0.001628
Epoch 1000/1000, Loss: 0.001323
------------------------------------------------------------
Inference complete! Mean loss: 0.001841
Shape of R_list: (1000, 1, 101, 100)
Shape of R_list after processing: (1000, 101, 100)
Finding fixed points for 1000 epochs...
  Epoch 100/1000: 0 fixed points
  Epoch 200/1000: 0 fixed points
  Epoch 300/1000: 0 fixed points
  Epoch 400/1000: 0 fixed points
  Epoch 500/1000: 0 fixed points
  Epoch 600/1000: 0 fixed points
  Epoch 700/1000: 0 fixed points
  Epoch 800/1000: 0 fixed points
  Epoch 900/1000: 0 fixed points
  Epoch 1000/1000: 0 fixed points


ValueError: n_components=2 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'

In [ ]:
rnn.reinitialize_weights(weights=['W_in', 'W_out', 'b_out'])

Reinitializing weights: ['W_in', 'W_out', 'b_out']


In [ ]:
# Train the RNN
train_R_list, losses, weight_snapshots = train_rnn(rnn, task, n_epochs=n_epochs, learning_rate=1e-2, batch_size=batch_size)
train_R_list = process_R_list(train_R_list)

_ = animate_fixed_points(rnn, train_R_list, save_path="plots/FP_training2_fullpca.mp4", stride=10, weight_snapshots=weight_snapshots)

Training RNN on flip flop task...
Epochs: 1000, Learning rate: 0.01, Batch size: 1
------------------------------------------------------------
Epoch 100/1000, Loss: 0.210329
Epoch 200/1000, Loss: 0.164822
Epoch 300/1000, Loss: 0.056897
Epoch 400/1000, Loss: 0.025257
Epoch 500/1000, Loss: 0.018018
Epoch 600/1000, Loss: 0.008449
Epoch 700/1000, Loss: 0.003048
Epoch 800/1000, Loss: 0.002201
Epoch 900/1000, Loss: 0.009211
Epoch 1000/1000, Loss: 0.002102
------------------------------------------------------------
Training complete!
Shape of R_list: (1000, 1, 101, 100)
Shape of R_list after processing: (1000, 101, 100)
Finding fixed points for 1000 epochs...
  Epoch 100/1000: 1 fixed points
  Epoch 200/1000: 1 fixed points
  Epoch 300/1000: 0 fixed points
  Epoch 400/1000: 0 fixed points
  Epoch 500/1000: 0 fixed points
  Epoch 600/1000: 0 fixed points
  Epoch 700/1000: 0 fixed points
  Epoch 800/1000: 0 fixed points
  Epoch 900/1000: 0 fixed points
  Epoch 1000/1000: 0 fixed points
Genera